In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib as mlp
import glob
from tqdm import tqdm
from collections import Counter
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')
from matplotlib import ticker

plt.style.use('seaborn-whitegrid')
sns.set_style("white",rc={
    'xtick.bottom': True,
    'ytick.left': True,
})
plt.rcParams['xtick.major.size'] = 5
plt.rcParams['ytick.major.size'] = 5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['ytick.major.width'] = 1

## Data

In [ ]:
polygons = # Series of strings, names of departments in the country.

Rij_vec = # Array of matrices, each matrix being the reconstructed reproduction operator in a certain interval of time. Nodes have polygons' order.

calendar = # Dates associated to the entries of Rij_vec. The first entry of calendar corresponds to the date of the first entry of Rij_vec.

pop_vec = # Array of populations, nodes have polygons' order. 

polygons_ind = # codes of polygons, French numbering. Example: Paris is department 75.

tot_pop = np.sum(pop_vec) 

M=len(polygons)

In [ ]:
vt_df=# DataFrame with columns 'Date', names of departments in the country, containing the equilibrium distribution for operators in that week
zt_df=# DataFrame with columns 'Date', names of departments in the country, containing v^* in that week
xt_df=# DataFrame with columns 'Date', names of departments in the country, containing spatial distribution of cases in that week (cases/total cases)
It_df=# DataFrame with columns 'Date', names of departments in the country, containing cases in that week

In [ ]:
zt_df.head()

In [ ]:
cases_daily= #DataFrame with columns 'Date', names of departments in the country, containing DAILY cases (non aggregated)

cases_daily.head()

# Projection on data

In [ ]:
# Select time intervals over which we compute average v*

z_new_int=['2020-11-17','2020-12-15']
z_old_int=['2020-10-20','2020-11-17']
z_old1_int=['2020-09-22','2020-10-20']
z_old2_int=['2020-08-25','2020-09-22']
z_old3_int=['2020-07-28','2020-08-25']
z_old4_int=['2020-06-30','2020-07-28']

# Compute average v*

z_new=zt_df.iloc[zt_df[zt_df['Date']==z_new_int[0]].index[0]:zt_df[zt_df['Date']==z_new_int[1]].index[0]+1].iloc[:,2:].mean(axis=0)
z_old=zt_df.iloc[zt_df[zt_df['Date']==z_old_int[0]].index[0]:zt_df[zt_df['Date']==z_old_int[1]].index[0]].iloc[:,2:].mean(axis=0)
z_old1=zt_df.iloc[zt_df[zt_df['Date']==z_old1_int[0]].index[0]:zt_df[zt_df['Date']==z_old1_int[1]].index[0]].iloc[:,2:].mean(axis=0)
z_old2=zt_df.iloc[zt_df[zt_df['Date']==z_old2_int[0]].index[0]:zt_df[zt_df['Date']==z_old2_int[1]].index[0]].iloc[:,2:].mean(axis=0)
z_old3=zt_df.iloc[zt_df[zt_df['Date']==z_old3_int[0]].index[0]:zt_df[zt_df['Date']==z_old3_int[1]].index[0]].iloc[:,2:].mean(axis=0)
z_old4=zt_df.iloc[zt_df[zt_df['Date']==z_old4_int[0]].index[0]:zt_df[zt_df['Date']==z_old4_int[1]].index[0]].iloc[:,2:].mean(axis=0)

# Check cosine similarity between obtained v*

print('1 - cos sim = '+str(1-np.dot(z_new,z_old)/(np.linalg.norm(z_new)*np.linalg.norm(z_old))))
print('1 - cos sim = '+str(1-np.dot(z_new,z_old1)/(np.linalg.norm(z_new)*np.linalg.norm(z_old1))))
print('1 - cos sim = '+str(1-np.dot(z_new,z_old2)/(np.linalg.norm(z_new)*np.linalg.norm(z_old2))))
print('1 - cos sim = '+str(1-np.dot(z_new,z_old3)/(np.linalg.norm(z_new)*np.linalg.norm(z_old3))))
print('1 - cos sim = '+str(1-np.dot(z_new,z_old4)/(np.linalg.norm(z_new)*np.linalg.norm(z_old4))))

In [ ]:
# Convert date format

rng=pd.to_datetime(pd.Series(z_new_int))
iso_rng=pd.Series([rng.iloc[i].isocalendar() for i in range(rng.shape[0])])
iso_rng_tp=pd.Series([str(iso_rng.iloc[i][0])+str(' W')+str(format(int(iso_rng.iloc[i][1]),'02d')) for i in range(rng.shape[0])])
print(iso_rng_tp)

In [ ]:
# Project daily cases on the obtained v*, then print on a csv

cases_daily_s=pd.DataFrame(data={'incidence':cases_daily.iloc[:,1:].sum(axis=1),'time':cases_daily['date']})
cases_daily_ws_new=pd.DataFrame(data={'incidence':cases_daily.iloc[:,1:].dot(z_new*len(polygons)),'time':cases_daily['date']})
cases_daily_ws_old=pd.DataFrame(data={'incidence':cases_daily.iloc[:,1:].dot(z_old*len(polygons)),'time':cases_daily['date']})
cases_daily_ws_old1=pd.DataFrame(data={'incidence':cases_daily.iloc[:,1:].dot(z_old1*len(polygons)),'time':cases_daily['date']})
cases_daily_ws_old2=pd.DataFrame(data={'incidence':cases_daily.iloc[:,1:].dot(z_old2*len(polygons)),'time':cases_daily['date']})
cases_daily_ws_old3=pd.DataFrame(data={'incidence':cases_daily.iloc[:,1:].dot(z_old3*len(polygons)),'time':cases_daily['date']})
cases_daily_ws_old4=pd.DataFrame(data={'incidence':cases_daily.iloc[:,1:].dot(z_old4*len(polygons)),'time':cases_daily['date']})

display(cases_daily_s[:2])
display(cases_daily_ws_new[:2])
display(cases_daily_ws_old[:2])   

cases_daily_s.to_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_cases.csv',sep=',')
cases_daily_ws_new.to_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_cases_ws_new.csv',sep=',')
cases_daily_ws_old.to_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_cases_ws_old.csv',sep=',')
cases_daily_ws_old1.to_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_cases_ws_old1.csv',sep=',')
cases_daily_ws_old2.to_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_cases_ws_old2.csv',sep=',')
cases_daily_ws_old3.to_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_cases_ws_old3.csv',sep=',')
cases_daily_ws_old4.to_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_cases_ws_old4.csv',sep=',')

Here we use a code in R. In particular, we use the R-package EpiEstim to read projected incidence and compute the reproduction ratio (with confidence intervals).
The generation interval distribution for COVID-19 is given as an input as well. In what follows, we elaborate the results. 

In [ ]:
S_df_epi = pd.read_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_epiestim.csv',sep=',',engine='python')
S_df_epi_zproj_new = pd.read_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_epiestim_ws_new.csv',sep=',',engine='python')
S_df_epi_zproj_old = pd.read_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_epiestim_ws_old.csv',sep=',',engine='python')
S_df_epi_zproj_old1 = pd.read_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_epiestim_ws_old1.csv',sep=',',engine='python')
S_df_epi_zproj_old2 = pd.read_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_epiestim_ws_old2.csv',sep=',',engine='python')
S_df_epi_zproj_old3 = pd.read_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_epiestim_ws_old3.csv',sep=',',engine='python')
S_df_epi_zproj_old4 = pd.read_csv('C:/Users/piero/Documents/PHD/eugenio/SIMULS/Fig4_epiestim_ws_old4.csv',sep=',',engine='python')

S_df_epi.insert(0,'date',np.array(cases_daily_s['time'][3:-4]),True)
S_df_epi_zproj_new.insert(0,'date',np.array(cases_daily_s['time'][3:-4]),True)
S_df_epi_zproj_old.insert(0,'date',np.array(cases_daily_s['time'][3:-4]),True)
S_df_epi_zproj_old1.insert(0,'date',np.array(cases_daily_s['time'][3:-4]),True)
S_df_epi_zproj_old2.insert(0,'date',np.array(cases_daily_s['time'][3:-4]),True)
S_df_epi_zproj_old3.insert(0,'date',np.array(cases_daily_s['time'][3:-4]),True)
S_df_epi_zproj_old4.insert(0,'date',np.array(cases_daily_s['time'][3:-4]),True)

S_df_epi['date'] = pd.to_datetime(S_df_epi['date'])
S_df_epi_zproj_new['date'] = pd.to_datetime(S_df_epi_zproj_new['date'])
S_df_epi_zproj_old['date'] = pd.to_datetime(S_df_epi_zproj_old['date'])
S_df_epi_zproj_old1['date'] = pd.to_datetime(S_df_epi_zproj_old1['date'])
S_df_epi_zproj_old2['date'] = pd.to_datetime(S_df_epi_zproj_old2['date'])
S_df_epi_zproj_old3['date'] = pd.to_datetime(S_df_epi_zproj_old3['date'])
S_df_epi_zproj_old4['date'] = pd.to_datetime(S_df_epi_zproj_old4['date'])

S_df_epi = S_df_epi.groupby([ pd.Grouper(key='date', freq='W-MON')])[['EE','lower','upper']].mean().reset_index().sort_values('date')
S_df_epi_zproj_new = S_df_epi_zproj_new.groupby([ pd.Grouper(key='date', freq='W-MON')])[['EE','lower','upper']].mean().reset_index().sort_values('date')
S_df_epi_zproj_old = S_df_epi_zproj_old.groupby([ pd.Grouper(key='date', freq='W-MON')])[['EE','lower','upper']].mean().reset_index().sort_values('date')
S_df_epi_zproj_old1 = S_df_epi_zproj_old1.groupby([ pd.Grouper(key='date', freq='W-MON')])[['EE','lower','upper']].mean().reset_index().sort_values('date')
S_df_epi_zproj_old2 = S_df_epi_zproj_old2.groupby([ pd.Grouper(key='date', freq='W-MON')])[['EE','lower','upper']].mean().reset_index().sort_values('date')
S_df_epi_zproj_old3 = S_df_epi_zproj_old3.groupby([ pd.Grouper(key='date', freq='W-MON')])[['EE','lower','upper']].mean().reset_index().sort_values('date')
S_df_epi_zproj_old4 = S_df_epi_zproj_old4.groupby([ pd.Grouper(key='date', freq='W-MON')])[['EE','lower','upper']].mean().reset_index().sort_values('date')

S_df_epi['date'] = pd.to_datetime(S_df_epi['date']) - pd.to_timedelta(6, unit='d')
S_df_epi_zproj_new['date'] = pd.to_datetime(S_df_epi_zproj_new['date']) - pd.to_timedelta(6, unit='d')
S_df_epi_zproj_old['date'] = pd.to_datetime(S_df_epi_zproj_old['date']) - pd.to_timedelta(6, unit='d')
S_df_epi_zproj_old1['date'] = pd.to_datetime(S_df_epi_zproj_old1['date']) - pd.to_timedelta(6, unit='d')
S_df_epi_zproj_old2['date'] = pd.to_datetime(S_df_epi_zproj_old2['date']) - pd.to_timedelta(6, unit='d')
S_df_epi_zproj_old3['date'] = pd.to_datetime(S_df_epi_zproj_old3['date']) - pd.to_timedelta(6, unit='d')
S_df_epi_zproj_old4['date'] = pd.to_datetime(S_df_epi_zproj_old4['date']) - pd.to_timedelta(6, unit='d')

display(S_df_epi[:2])
display(S_df_epi_zproj_new[:2])
display(S_df_epi_zproj_old[:2])

In [ ]:
start=S_df_epi[S_df_epi['date']=='2020-12-15'].index[0]
stop=S_df_epi[S_df_epi['date']=='2021-03-30'].index[0]
dates=pd.to_datetime(S_df_epi.iloc[start:stop].loc[:,'date'])
iso_dates=pd.Series([dates.iloc[i].isocalendar() for i in range(dates.shape[0])])
iso_dates_tp=pd.Series([str(iso_dates.iloc[i][0])+str(' W')+str(format(int(iso_dates.iloc[i][1]),'02d')) for i in range(dates.shape[0])])

In [ ]:
# Plot

fig,ax=plt.subplots(figsize=(10,5),dpi=300)

cp=sns.color_palette('Blues',6)
ax.axhline(1,0,1,color='dimgrey',linestyle='dashed',alpha=1)

ax.plot(iso_dates_tp,S_df_epi['EE'][start:stop],label='uncorrected',color='orchid')
ax.fill_between(iso_dates_tp,S_df_epi['lower'][start:stop],S_df_epi['upper'][start:stop],color='orchid',alpha=.3)

ax.plot(iso_dates_tp,S_df_epi_zproj_new['EE'][start:stop],label=r'W47-W50',color=cp[-1])
ax.plot(iso_dates_tp,S_df_epi_zproj_old['EE'][start:stop],label=r'W43-W46',color=cp[-2])
ax.plot(iso_dates_tp,S_df_epi_zproj_old1['EE'][start:stop],label=r'W39-W42',color=cp[-3])
ax.plot(iso_dates_tp,S_df_epi_zproj_old2['EE'][start:stop],label=r'W35-W38',color=cp[-4])
ax.plot(iso_dates_tp,S_df_epi_zproj_old3['EE'][start:stop],label=r'W31-W34',color=cp[-5])
ax.fill_between(iso_dates_tp,S_df_epi_zproj_new['lower'][start:stop],S_df_epi_zproj_new['upper'][start:stop],color=cp[-1],alpha=.3)

ax.set_ylim([0.7,1.3])
ax.set_xlim(['2020 W51','2021 W12'])
ax.tick_params(axis='x',labelsize=15,labelrotation=90)
ax.tick_params(axis='y',labelsize=15)
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter("{x:01.2f}"))
ax.set_ylabel('Reproduction ratio',fontsize=15)
_=ax.legend(fontsize=15,frameon=(True),ncol=3,bbox_to_anchor=(.99,0.265))
fig.tight_layout()